In [70]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [71]:
import pymongo
from pymongo import MongoClient
from odo import odo
import warnings
warnings.filterwarnings('ignore')
import os
import json

In [72]:
path = ['data','paragraphs','qas','answers']

In [73]:
#original = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()))
global original
curr_dir = os.getcwd()
with open(curr_dir+'/train-v1.1.json', 'r') as read_training:
    original = json.load(read_training)
    #original = json.dumps(read_training)
js = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path)
m = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path[:-1])
r = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path[:-2])

In [74]:
dataframe = pd.read_csv("train_detect_sent.csv").reset_index(drop=True)

In [75]:
dataframe = dataframe[dataframe["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [76]:
dataframe.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc,root_match_idx,root_match_idx_first
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519995, 0.0501314 , 0.04787038,...",[[ 0.10956341 0.1142294 0.04428942 ... 0.0...,"[0.43262481689453125, 0.3686484098434448, 0.36...","[14.108409, 15.0115595, 18.043663, 13.160973, ...",5,5,[5],5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519995, 0.0501314 , 0.04787038,...",[[ 0.10951652 0.11030623 0.05210007 ... -0.0...,"[0.45407456159591675, 0.32262009382247925, 0.3...","[12.889506, 12.285218, 16.843704, 8.361172, 11...",3,3,[],0
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519995, 0.0501314 , 0.04787038,...",[[ 0.00262268 0.14930709 0.03063984 ... 0.0...,"[0.39596283435821533, 0.315723180770874, 0.346...","[11.590836, 12.180486, 16.56459, 7.4446793, 9....",3,3,"[1, 2, 3, 4, 5, 6]",1


In [77]:
ast.literal_eval(dataframe["euclidean_dis"][0])

[14.108409, 15.0115595, 18.043663, 13.160973, 12.855492, 9.627245, 15.943851]

In [78]:
def feature_set_cos_euc(dataframe):
    train_data = pd.DataFrame()
    
    for k in range(len(dataframe["cosine_sim"])):
        cosine_dist = ast.literal_eval(dataframe["cosine_sim"][k].replace("nan","1"))#.replace('NaN', '1'))
        for i in range(len(cosine_dist)):
            train_data.loc[k, "cos_"+"%s"%i] = cosine_dist[i]
            
    for k in range(len(dataframe["euclidean_dis"])):
        eucl_dis = ast.literal_eval(dataframe["euclidean_dis"][k])
        for i in range(len(eucl_dis)):
            train_data.loc[k, "euc_"+"%s"%i] = eucl_dis[i]
    
            
    train_data["target"] = dataframe["target"]
    return train_data

In [79]:
train_data = feature_set_cos_euc(dataframe)

In [80]:
#train_data.fillna(1, inplace=True)

In [81]:
train_data.head(20)

,cos_0,cos_1,cos_2,cos_3,cos_4,cos_5,cos_6,cos_7,cos_8,cos_9,...,euc_1,euc_2,euc_3,euc_4,euc_5,euc_6,euc_7,euc_8,euc_9,target
0,0.432625,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,NaN,NaN,NaN,...,15.011560,18.043663,13.160973,12.855492,9.627245,15.943851,NaN,NaN,NaN,5
1,0.454075,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,NaN,NaN,NaN,...,12.285218,16.843704,8.361172,11.918098,17.601220,14.929260,NaN,NaN,NaN,2
2,0.395963,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,NaN,NaN,NaN,...,12.180486,16.564590,7.444679,9.165415,14.745420,13.290999,NaN,NaN,NaN,3
3,0.490070,0.406061,0.456177,0.353780,0.373973,0.368208,0.450643,NaN,NaN,NaN,...,15.017246,20.812680,10.511387,10.947037,16.777027,17.992474,NaN,NaN,NaN,4
4,0.477764,0.312099,0.365222,0.272937,0.406829,0.378837,0.370091,NaN,NaN,NaN,...,12.130126,17.447662,8.731177,12.842587,17.582539,15.477146,NaN,NaN,NaN,1
5,0.254865,0.307463,0.343259,0.324606,0.361211,0.397475,NaN,NaN,NaN,NaN,...,12.555490,16.011751,12.947239,11.467085,15.898802,NaN,NaN,NaN,NaN,0
6,0.268173,0.298206,0.335387,0.313678,0.300063,0.328335,NaN,NaN,NaN,NaN,...,12.238684,15.715960,12.581003,9.624770,13.433640,NaN,NaN,NaN,NaN,5
7,0.352783,0.332142,0.318186,0.387593,0.362175,0.352322,NaN,NaN,NaN,NaN,...,13.846157,15.199486,15.804577,12.319434,14.740927,NaN,NaN,NaN,NaN,2
8,0.423420,0.377198,0.385242,0.362514,0.411731,0.346209,NaN,NaN,NaN,NaN,...,15.616562,18.053150,14.824701,13.995485,14.497253,NaN,NaN,NaN,NaN,3
9,0.569263,0.541039,0.582341,0.527196,0.340677,0.392760,NaN,NaN,NaN,NaN,...,20.027370,24.360714,19.273872,10.326596,15.372666,NaN,NaN,NaN,NaN,5


In [82]:
train_data.apply(max, axis = 0)

cos_0      0.927659
cos_1      1.008395
cos_2      0.869930
cos_3      0.929962
cos_4      0.985284
cos_5      1.000000
cos_6      1.000000
cos_7           NaN
cos_8           NaN
cos_9           NaN
euc_0     50.308200
euc_1     33.063713
euc_2     44.460900
euc_3     34.358498
euc_4     30.521135
euc_5     29.758953
euc_6     37.077620
euc_7           NaN
euc_8           NaN
euc_9           NaN
target     9.000000
dtype: float64

In [83]:
train_data.iloc[:,0].mean()


0.33452995352365156

In [84]:
sh = train_data.shape
def nan_to_average(train_data):
    for i in range(sh[1]): 
        #for j in range(sh[0]):
        #train_data.iloc[:,i].fillna(train_data.iloc[:,i].mean(), inplace=True)
        train_data.iloc[:,i].fillna(np.max(train_data.iloc[:,i]), inplace=True)
    return train_data

In [85]:
train_data = nan_to_average(train_data)

In [86]:
train_data.head(20)

,cos_0,cos_1,cos_2,cos_3,cos_4,cos_5,cos_6,cos_7,cos_8,cos_9,...,euc_1,euc_2,euc_3,euc_4,euc_5,euc_6,euc_7,euc_8,euc_9,target
0,0.432625,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,1.0,0.503029,0.474452,...,15.011560,18.043663,13.160973,12.855492,9.627245,15.943851,24.510351,28.98883,20.888351,5
1,0.454075,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,1.0,0.503029,0.474452,...,12.285218,16.843704,8.361172,11.918098,17.601220,14.929260,24.510351,28.98883,20.888351,2
2,0.395963,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,1.0,0.503029,0.474452,...,12.180486,16.564590,7.444679,9.165415,14.745420,13.290999,24.510351,28.98883,20.888351,3
3,0.490070,0.406061,0.456177,0.353780,0.373973,0.368208,0.450643,1.0,0.503029,0.474452,...,15.017246,20.812680,10.511387,10.947037,16.777027,17.992474,24.510351,28.98883,20.888351,4
4,0.477764,0.312099,0.365222,0.272937,0.406829,0.378837,0.370091,1.0,0.503029,0.474452,...,12.130126,17.447662,8.731177,12.842587,17.582539,15.477146,24.510351,28.98883,20.888351,1
5,0.254865,0.307463,0.343259,0.324606,0.361211,0.397475,1.000000,1.0,0.503029,0.474452,...,12.555490,16.011751,12.947239,11.467085,15.898802,37.077620,24.510351,28.98883,20.888351,0
6,0.268173,0.298206,0.335387,0.313678,0.300063,0.328335,1.000000,1.0,0.503029,0.474452,...,12.238684,15.715960,12.581003,9.624770,13.433640,37.077620,24.510351,28.98883,20.888351,5
7,0.352783,0.332142,0.318186,0.387593,0.362175,0.352322,1.000000,1.0,0.503029,0.474452,...,13.846157,15.199486,15.804577,12.319434,14.740927,37.077620,24.510351,28.98883,20.888351,2
8,0.423420,0.377198,0.385242,0.362514,0.411731,0.346209,1.000000,1.0,0.503029,0.474452,...,15.616562,18.053150,14.824701,13.995485,14.497253,37.077620,24.510351,28.98883,20.888351,3
9,0.569263,0.541039,0.582341,0.527196,0.340677,0.392760,1.000000,1.0,0.503029,0.474452,...,20.027370,24.360714,19.273872,10.326596,15.372666,37.077620,24.510351,28.98883,20.888351,5


In [87]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train_data.iloc[:,:-1])

In [88]:
train_x, test_x, train_y, test_y = train_test_split(X,
train_data.iloc[:,-1], train_size=0.8, random_state = 5)

In [89]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))



Multinomial Logistic regression Train Accuracy :  0.5412087912087912
Multinomial Logistic regression Test Accuracy :  0.510989010989011


### Logistic regression

In [90]:
training_data = pd.read_csv("train_detect_sent.csv").reset_index(drop=True)

In [91]:
training_data = training_data[training_data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [92]:
def get_columns_from_root(train):
    
    for i in range(train.shape[0]):
        if len(ast.literal_eval(train["root_match_idx"][i])) == 0: pass
        
        else:
            for item in ast.literal_eval(train["root_match_idx"][i]):
                train.loc[i, "column_root_"+"%s"%item] = 1
    return train

In [93]:
training_data = get_columns_from_root(training_data)

In [94]:
training_data.head(3).transpose()

,0,1,2
context,"Architecturally, the school has a Catholic cha...","Architecturally, the school has a Catholic cha...","Architecturally, the school has a Catholic cha..."
question,To whom did the Virgin Mary allegedly appear i...,What is in front of the Notre Dame Main Building?,The Basilica of the Sacred heart at Notre Dame...
answer_start,515,188,279
text,Saint Bernadette Soubirous,a copper statue of Christ,the Main Building
sentences,"['Architecturally, the school has a Catholic c...","['Architecturally, the school has a Catholic c...","['Architecturally, the school has a Catholic c..."
target,5,2,3
sent_emb,"[array([ 0.05519995, 0.0501314 , 0.04787038,...","[array([ 0.05519995, 0.0501314 , 0.04787038,...","[array([ 0.05519995, 0.0501314 , 0.04787038,..."
quest_emb,[[ 0.10956341 0.1142294 0.04428942 ... 0.0...,[[ 0.10951652 0.11030623 0.05210007 ... -0.0...,[[ 0.00262268 0.14930709 0.03063984 ... 0.0...
cosine_sim,"[0.43262481689453125, 0.3686484098434448, 0.36...","[0.45407456159591675, 0.32262009382247925, 0.3...","[0.39596283435821533, 0.315723180770874, 0.346..."
euclidean_dis,"[14.108409, 15.0115595, 18.043663, 13.160973, ...","[12.889506, 12.285218, 16.843704, 8.361172, 11...","[11.590836, 12.180486, 16.56459, 7.4446793, 9...."


In [95]:
sset = training_data[["column_root_0","column_root_1","column_root_2","column_root_3","column_root_4","column_root_5",\
             "column_root_6","column_root_7","column_root_8","column_root_9"]]

In [96]:
sset.fillna(0, inplace=True)

In [97]:
sset.head(3)

,column_root_0,column_root_1,column_root_2,column_root_3,column_root_4,column_root_5,column_root_6,column_root_7,column_root_8,column_root_9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [98]:
train_data.columns


Index(['cos_0', 'cos_1', 'cos_2', 'cos_3', 'cos_4', 'cos_5', 'cos_6', 'cos_7',
       'cos_8', 'cos_9', 'euc_0', 'euc_1', 'euc_2', 'euc_3', 'euc_4', 'euc_5',
       'euc_6', 'euc_7', 'euc_8', 'euc_9', 'target'],
      dtype='object')

In [99]:
cset = pd.concat([sset, train_data],axis=1, join_axes=[sset.index])

In [100]:
cset.head(3)

,column_root_0,column_root_1,column_root_2,column_root_3,column_root_4,column_root_5,column_root_6,column_root_7,column_root_8,column_root_9,...,euc_1,euc_2,euc_3,euc_4,euc_5,euc_6,euc_7,euc_8,euc_9,target
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.011560,18.043663,13.160973,12.855492,9.627245,15.943851,24.510351,28.98883,20.888351,5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.285218,16.843704,8.361172,11.918098,17.601220,14.929260,24.510351,28.98883,20.888351,2
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,12.180486,16.564590,7.444679,9.165415,14.745420,13.290999,24.510351,28.98883,20.888351,3


In [101]:
cset.columns

Index(['column_root_0', 'column_root_1', 'column_root_2', 'column_root_3',
       'column_root_4', 'column_root_5', 'column_root_6', 'column_root_7',
       'column_root_8', 'column_root_9', 'cos_0', 'cos_1', 'cos_2', 'cos_3',
       'cos_4', 'cos_5', 'cos_6', 'cos_7', 'cos_8', 'cos_9', 'euc_0', 'euc_1',
       'euc_2', 'euc_3', 'euc_4', 'euc_5', 'euc_6', 'euc_7', 'euc_8', 'euc_9',
       'target'],
      dtype='object')

In [102]:
train_x, test_x, train_y, test_y = train_test_split(cset.iloc[:,:-1],
cset.iloc[:,-1], train_size=0.8, random_state = 5)

In [103]:
linear_reagression = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
linear_reagression.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [104]:
print("Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, linear_reagression.predict(train_x)))
print("Logistic regression regression Test Accuracy : ", metrics.accuracy_score(test_y, linear_reagression.predict(test_x)))

Logistic regression Train Accuracy :  0.5302197802197802
Logistic regression regression Test Accuracy :  0.4835164835164835


In [105]:
print("Logistic regression Train F1 score : ", metrics.f1_score(train_y, linear_reagression.predict(train_x), average='weighted'))
print("Logistic regression Test F1 score : ", metrics.f1_score(test_y, linear_reagression.predict(test_x), average='weighted'))

Logistic regression Train F1 score :  0.5196608419710219
Logistic regression Test F1 score :  0.463171363747165


In [106]:
print(metrics.classification_report(train_y, linear_reagression.predict(train_x)))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         5
          0       0.53      0.68      0.60       190
          1       0.58      0.62      0.60       149
          2       0.54      0.42      0.47       127
          3       0.48      0.38      0.43        91
          4       0.45      0.31      0.37        64
          5       0.48      0.51      0.49        45
          6       0.30      0.26      0.28        27
          7       0.68      0.87      0.76        15
          8       0.71      0.77      0.74        13
          9       1.00      1.00      1.00         2

avg / total       0.52      0.53      0.52       728



--- 

### Random Forest

In [110]:
rand_forest = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rand_forest.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rand_forest.predict(train_x)))
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, rand_forest.predict(test_x)))

Random Forest Train Accuracy :  0.7582417582417582
Random Forest Test Accuracy :  0.5384615384615384


In [111]:
print("Random Forest Train F1 score : ", metrics.f1_score(train_y, rand_forest.predict(train_x), average='weighted'))
print("Random Forest Test F1 score : ", metrics.f1_score(test_y, rand_forest.predict(test_x), average='weighted'))

Random Forest Train F1 score :  0.7486144334433332
Random Forest Test F1 score :  0.5194671302317179


In [112]:
print(metrics.classification_report(train_y, rand_forest.predict(train_x)))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         5
          0       0.70      0.89      0.79       190
          1       0.76      0.90      0.82       149
          2       0.80      0.69      0.74       127
          3       0.76      0.67      0.71        91
          4       0.82      0.64      0.72        64
          5       0.79      0.67      0.72        45
          6       0.85      0.41      0.55        27
          7       0.87      0.87      0.87        15
          8       0.86      0.46      0.60        13
          9       0.00      0.00      0.00         2

avg / total       0.76      0.76      0.75       728

